# Eager Mode (Aplpa) 

Eager Mode is an interactive way to build superduper applications. 

Users can input data as usual, continuously call models, and view results.
Once the interactive debugging and construction are complete, 
the corresponding data pipeline can be built directly through `apply`, 
eliminating the need for direct debugging between AI application models and databases.

:::note
This feature is in the alpha version.
:::

Connect the database and insert data.

In [1]:
from superduper import superduper

db = superduper('mongomock://test')

2024-Jul-25 11:20:56.95| INFO     | zhouhaha-2.local| superduper.base.build:57   | Data Client is ready. mongomock.MongoClient('localhost', 27017)
2024-Jul-25 11:20:56.96| INFO     | zhouhaha-2.local| superduper.base.build:36   | Connecting to Metadata Client with engine:  mongomock.MongoClient('localhost', 27017)
2024-Jul-25 11:20:56.96| INFO     | zhouhaha-2.local| superduper.base.build:152  | Connecting to compute client: Compute(uri=None, compute_kwargs={}, _path='superduper.backends.local.compute.LocalComputeBackend')
2024-Jul-25 11:20:56.96| INFO     | zhouhaha-2.local| superduper.base.datalayer:105  | Building Data Layer
2024-Jul-25 11:20:56.96| INFO     | zhouhaha-2.local| superduper.base.build:214  | Configuration: 
 +---------------+------------------+
| Configuration |      Value       |
+---------------+------------------+
|  Data Backend | mongomock://test |
+---------------+------------------+


In [2]:
import numpy as np
data = [
    {"x": 1, "y": "2", "z": np.array([1, 2, 3])},
    {"x": 2, "y": "3", "z": np.array([4, 5, 6])},
    {"x": 3, "y": "4", "z": np.array([7, 8, 9])},
]

db["documents"].insert(data).execute()

2024-Jul-25 11:20:57.00| INFO     | zhouhaha-2.local| superduper.base.datalayer:426  | Table documents does not exist, auto creating...
2024-Jul-25 11:20:57.00| INFO     | zhouhaha-2.local| superduper.base.datalayer:432  | Creating table documents with schema {('z', 'numpy-int64[3]')}
2024-Jul-25 11:20:57.00| WARNING  | zhouhaha-2.local| superduper.misc.annotations:119  | add is deprecated and will be removed in a future release.


([ObjectId('66a1c49909d51b458635eda2'),
  ObjectId('66a1c49909d51b458635eda3'),
  ObjectId('66a1c49909d51b458635eda4')],
 None)

When using `select.execute(eager_mode=True)`, all returned data will enter eager mode, which can be used for interactive model pipeline construction.

In [3]:
data = list(db["documents"].select().execute(eager_mode=True))[0]
data

Document({'x': 1, 'y': '2', 'z': array([1, 2, 3]), '_fold': 'train', '_id': ObjectId('66a1c49909d51b458635eda2')})

Define the first model and make predictions.

In [4]:
from superduper import ObjectModel
def func_a(x):
    return {"x": x, "model": "a"}

model_a = ObjectModel(identifier="a", object=func_a)
output_a = model_a(data["x"])
output_a

{'x': 1, 'model': 'a'}

Define the second model and make predictions.

In [5]:
def func_b(x, y, o_a):
    return {
        "x": x,
        "y": y,
        "o_a": o_a,
        "model": "b"
    }

model_b = ObjectModel(identifier="b", object=func_b)
output_b = model_b(data["x"], data["y"], output_a)
output_b

{'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}

Define the third model and make predictions.

In [6]:
def func_c(x, y, z, o_a, o_b):
    return {
        "x": x,
        "y": y,
        "z": z,
        "o_a": o_a,
        "o_b": o_b,
        "model": "c",
    }

model_c = ObjectModel(identifier="c", object=func_c)
output_c = model_c(data["x"], data["y"], data["z"], output_a, output_b)
output_c

{'x': 1, 'y': '2', 'z': array([1, 2, 3]), 'o_a': {'x': 1, 'model': 'a'}, 'o_b': {'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}, 'model': 'c'}

Apply all models to the data to start monitoring the data and making predictions.
When adding a model result, not only the current model but also the recursively dependent upstream models will be added.

In [7]:
output_c.apply()

2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:289  | Applying nodes: ['a', 'b', 'c']
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:301  | Applying node: a
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:302  | Example output {'x': 1, 'model': 'a'}
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:306  | Key: x
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:309  | Select: documents.find({}, {'x': 1, '_builds': 1, '_files': 1, '_blobs': 1, '_schema': 1})
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:311  | Predict id: a
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:313  | Predict kwargs: {}
2024-Jul-25 11:20:57.14| INFO     | zhouhaha-2.local| superduper.misc.eager:323  | Listener: Listener(identifier='a', uuid='a', upstream=None, plugins=None, key='x', model=ObjectModel(identifier='a', u

In [8]:
list(db["documents"].select().outputs("a", "b", "c").select().execute())

[Document({'_id': ObjectId('66a1c49909d51b458635eda2'), '_outputs': {'c': {'x': 1, 'y': '2', 'z': Encodable(identifier='e2d1839ed1706f7d470d87f8c48a5584cafa5a12', uuid='6ec1ba5e-0b93-49a0-a362-1637fd933849', datatype=DataType(identifier='numpy-int64[3]', uuid='72357574-0c3e-4503-b0c7-e15a6595491d', upstream=None, plugins=None, encoder=<superduper.ext.numpy.encoder.EncodeArray object at 0x146f805b0>, decoder=<superduper.ext.numpy.encoder.DecodeArray object at 0x146f820b0>, info=None, shape=[3], directory=None, encodable='encodable', bytes_encoding=<BytesEncoding.BYTES: 'Bytes'>, intermediate_type='bytes', media_type=None), uri=None, x=array([1, 2, 3])), 'o_a': {'x': 1, 'model': 'a'}, 'o_b': {'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}, 'model': 'c'}, 'b': {'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}, 'a': {'x': 1, 'model': 'a'}}}),
 Document({'_id': ObjectId('66a1c49909d51b458635eda3'), '_outputs': {'c': {'x': 2, 'y': '3', 'z': Encodable(identifier=

If you want to modify the predict_id of a specific model, 
you can use `output.predict_id = "your_predict_id"` to set it.

In [9]:
model_predict_id = ObjectModel(identifier="c", object=func_c)
output_predict_id = model_predict_id(data["x"], data["y"], data["z"], output_a, output_b)
output_predict_id.predict_id = "new_predict_id"
output_predict_id.apply()

2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:289  | Applying nodes: ['a', 'b', 'new_predict_id']
2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:298  | Node [a] already applied.
2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:298  | Node [b] already applied.
2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:301  | Applying node: new_predict_id
2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:302  | Example output {'x': 1, 'y': '2', 'z': array([1, 2, 3]), 'o_a': {'x': 1, 'model': 'a'}, 'o_b': {'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}, 'model': 'c'}
2024-Jul-25 11:20:57.21| INFO     | zhouhaha-2.local| superduper.misc.eager:306  | Key: ('x', 'y', 'z', '_outputs.a', '_outputs.b')
2024-Jul-25 11:20:57.22| INFO     | zhouhaha-2.local| superduper.misc.eager:309  | Select: MongoOutputs(identifier='MongoOutputs-identifier-uuid-0e7da99

View the prediction results of all data in the database.

In [10]:
list(db["_outputs.new_predict_id"].select().execute())

[Document({'_outputs': {'new_predict_id': {'x': 1, 'y': '2', 'z': Encodable(identifier='e2d1839ed1706f7d470d87f8c48a5584cafa5a12', uuid='1c2c16b6-66cf-414d-a0f7-e8702df24569', datatype=DataType(identifier='numpy-int64[3]', uuid='565196cb-fc5e-41f5-a293-8bb6db908046', upstream=None, plugins=None, encoder=<superduper.ext.numpy.encoder.EncodeArray object at 0x146f80910>, decoder=<superduper.ext.numpy.encoder.DecodeArray object at 0x146f81240>, info=None, shape=[3], directory=None, encodable='encodable', bytes_encoding=<BytesEncoding.BYTES: 'Bytes'>, intermediate_type='bytes', media_type=None), uri=None, x=array([1, 2, 3])), 'o_a': {'x': 1, 'model': 'a'}, 'o_b': {'x': 1, 'y': '2', 'o_a': {'x': 1, 'model': 'a'}, 'model': 'b'}, 'model': 'c'}}, '_source': ObjectId('66a1c49909d51b458635eda2'), '_fold': 'train', '_id': ObjectId('66a1c49909d51b458635edc0')}),
 Document({'_outputs': {'new_predict_id': {'x': 2, 'y': '3', 'z': Encodable(identifier='168d0ba38783ef14943e28073c750dbd4b83bdcc', uuid='c

If you want to perform if-like conditional operations to route data using different models, you can use `set_condition` to handle it. Currently, only equals and not equals conditions are supported.

In [11]:
model_condition = ObjectModel(identifier="condition", object=func_a)
output_condition = model_condition(data["x"])
output_condition.set_condition(data["x"] == 1)
output_condition.apply()
output_condition

2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:289  | Applying nodes: ['condition']
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:301  | Applying node: condition
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:302  | Example output {'x': 1, 'model': 'a'}
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:306  | Key: x
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:309  | Select: documents.find({'x': 1}, {'x': 1, '_builds': 1, '_files': 1, '_blobs': 1, '_schema': 1})
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:311  | Predict id: condition
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:313  | Predict kwargs: {}
2024-Jul-25 11:20:57.27| INFO     | zhouhaha-2.local| superduper.misc.eager:323  | Listener: Listener(identifier='condition', uuid='condition', upstream=None, plugins=None, key='x',

{'x': 1, 'model': 'a'}

In [12]:
db['documents'].find({}, {'x': 1, '_builds': 1, '_files': 1, '_blobs': 1, '_schema': 1}).filter({'x': 1}).execute()

SuperDuperCursor(raw_cursor=<mongomock.collection.Cursor object at 0x146f826b0>, id_field='_id', db=<superduper.base.datalayer.Datalayer object at 0x146f38f70>, scores=None, schema=None, process_func=None, _it=0)

In [13]:
list(db["_outputs.condition"].select().execute())

[Document({'_outputs': {'condition': {'x': 1, 'model': 'a'}}, '_source': ObjectId('66a1c49909d51b458635eda2'), '_fold': 'train', '_id': ObjectId('66a1c49909d51b458635edc7')})]